In [1]:
import pandas as pd
from collections import defaultdict
import re
import numpy as np
import sys
sys.path.insert(0, '../../base_codes')
sys.path.insert(0, '../../data_specific_codes')
sys.path.insert(0, '../../utility_codes')

#%run -i 'parameters'
#%run -i 'main_functions'
#%run -i 'utility_functions'

from RE_init import *
from main_functions import *
from utility_functions import *

##MICE_AND_MEN
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Mice_and_men/'
based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/mice_and_men/'
file_input_name = 'mice_and_men_raw_relations_-1.csv'
input_name_prefix = 'mice_and_men_raw'
DATA_SET = 'goodreads-Mice-and-men'

##HOBBIT
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Hobbit/'
#based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/hobbit/'
#file_input_name= 'hobbit_raw_relations_-1.csv'
#input_name_prefix = 'hobbit_raw'
#DATA_SET = 'goodreads-Hobbit'

##Mockingbird
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Mockingbird/'
#based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/mockingbird/'
#file_input_name = 'mockingbird_raw_relations_-1.csv'
#input_name_prefix = 'mockingbird_raw'
#DATA_SET = 'goodreads-Mockingbird'

#Frankenstein
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Frankenstein/'
#based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/frankenstein/'
#file_input_name = 'frankenstein_raw_relations_-1.csv'
#input_name_prefix = 'frankenstein'
#DATA_SET = 'goodreads-Frankenstein'

def read_df_rel(based_dir, file_input_name):
    file_input = based_dir + file_input_name    
    ff = open(file_input)
    delim=","
    df = pd.read_csv(file_input,delimiter=delim,header=0)        
    return df

df_rels = read_df_rel(based_dir, file_input_name)

In [21]:
df_rels

,post_num,sentence_num,sentence,arg1,rel,arg2,type,pattern,arg1_with_pos,rel_with_pos,arg2_with_pos,arg1_prepositions,rel_prepositions,arg2_prepositions,annotation
0,0,0,It's the way Steinbeck describes things that g...,{Steinbeck},{describes},{things},SVO,"(nsubj, verb, dobj)",Steinbeck-NNP-5,describes-VBZ-6,things-NNS-7,NaN,NaN,NaN,"{'dep_parse': ""nsubj(way-4, It-1)\ncop(way-4, ..."
1,0,0,It's the way Steinbeck describes things that g...,{things},{gets},{me.},SVO,"(nsubj, verb, dobj)",things-NNS-7,gets-VBZ-9,me.-.-10,NaN,NaN,NaN,"{'dep_parse': ""nsubj(way-4, It-1)\ncop(way-4, ..."
2,0,1,"Crooks, the negro stable buck, had his bunk in...",{Crooks},{had},{bunk},SVO,"(nsubj, verb, dobj)",Crooks-NNS-1,had-VBD-8,bunk-NN-10,NaN,NaN,REASON: prep_in TEXT: the harness {room} --,"{'dep_parse': 'nsubj(had-8, Crooks-1)\ndet(buc..."
3,0,1,"Crooks, the negro stable buck, had his bunk in...",a little {shed},{leaned} off,the {wall} barn,SVO,"(nsubj, verb, dobj)",a-DT-16 little-JJ-17 shed-NN-18,leaned-VBD-20 off-RP-21,the-DT-22 wall-NN-23 barn-NN-26,NaN,NaN,NaN,"{'dep_parse': 'nsubj(had-8, Crooks-1)\ndet(buc..."
4,0,7,"On pegs were also pieces of harness, a split c...",{leather},{covering},{split},SVO,"(nsubj, verb, dobj)",leather-NN-29,covering-VBG-30,split-NN-31,NaN,NaN,NaN,"{'dep_parse': 'prep_on(pieces-5, pegs-2)\ncop(..."
5,2,6,Proof the incredibly sad ending is not hopeles...,{I},could still {hope},{it},SVO,"(nsubj, verb, dobj)",I-PRP-14,could-MD-15 still-RB-16 hope-VB-17,it-PRP-18,NaN,REASON: prep_on TEXT: a {reread} --,NaN,"{'dep_parse': 'csubj(is-9, Proof-1)\ndet(endin..."
6,2,7,"Oh, it does happen, loneliness and cruelty and...",{it},does {happen},loneliness cruelty mob hating {mentality},SVO,"(nsubj, verb, dobj)",it-PRP-3,does-VBZ-4 happen-VB-5,loneliness-NN-7 cruelty-NN-9 mob-NN-12 hating-...,NaN,NaN,NaN,"{'dep_parse': 'discourse(happen-5, Oh-1)\nnsub..."
7,2,8,"The possibility that it won't, that people mig...",{it},{won},{'t},SVO,"(nsubj, verb, dobj)",it-PRP-4,won-VBD-5,'t-NN-6,NaN,NaN,NaN,"{'dep_parse': ""det(possibility-2, The-1)\nnsub..."
8,2,10,Lennie dying alone like a dog haunts me.,a {dog},{haunts},{me.},SVO,"(nsubj, verb, dobj)",a-DT-5 dog-NN-6,haunts-VBZ-7,me.-.-8,NaN,NaN,NaN,"{'dep_parse': 'root(ROOT-0, Lennie-1)\namod(Le..."
9,2,12,"Dignity, yes, because in this case the dying w...",{the} dying,was {robbing},{life} dignity,SVO,"(nsubj, verb, dobj)",the-DT-9 dying-VBG-10,was-VBD-11 robbing-VBG-12,life-NN-14 dignity-NN-16,NaN,REASON: prep_in TEXT: this {case} --,NaN,"{'dep_parse': 'root(ROOT-0, Dignity-1)\ndiscou..."


In [22]:
df = df_rels[["sentence","arg1", "rel", "arg2", "arg1_prepositions","rel_prepositions","arg2_prepositions"]]

In [23]:
df

,sentence,arg1,rel,arg2,arg1_prepositions,rel_prepositions,arg2_prepositions
0,It's the way Steinbeck describes things that g...,{Steinbeck},{describes},{things},NaN,NaN,NaN
1,It's the way Steinbeck describes things that g...,{things},{gets},{me.},NaN,NaN,NaN
2,"Crooks, the negro stable buck, had his bunk in...",{Crooks},{had},{bunk},NaN,NaN,REASON: prep_in TEXT: the harness {room} --
3,"Crooks, the negro stable buck, had his bunk in...",a little {shed},{leaned} off,the {wall} barn,NaN,NaN,NaN
4,"On pegs were also pieces of harness, a split c...",{leather},{covering},{split},NaN,NaN,NaN
5,Proof the incredibly sad ending is not hopeles...,{I},could still {hope},{it},NaN,REASON: prep_on TEXT: a {reread} --,NaN
6,"Oh, it does happen, loneliness and cruelty and...",{it},does {happen},loneliness cruelty mob hating {mentality},NaN,NaN,NaN
7,"The possibility that it won't, that people mig...",{it},{won},{'t},NaN,NaN,NaN
8,Lennie dying alone like a dog haunts me.,a {dog},{haunts},{me.},NaN,NaN,NaN
9,"Dignity, yes, because in this case the dying w...",{the} dying,was {robbing},{life} dignity,NaN,REASON: prep_in TEXT: this {case} --,NaN


In [27]:
df_with_prep = df.dropna(axis=0, thresh=5)

In [28]:
df_with_prep

,sentence,arg1,rel,arg2,arg1_prepositions,rel_prepositions,arg2_prepositions
2,"Crooks, the negro stable buck, had his bunk in...",{Crooks},{had},{bunk},NaN,NaN,REASON: prep_in TEXT: the harness {room} --
5,Proof the incredibly sad ending is not hopeles...,{I},could still {hope},{it},NaN,REASON: prep_on TEXT: a {reread} --,NaN
9,"Dignity, yes, because in this case the dying w...",{the} dying,was {robbing},{life} dignity,NaN,REASON: prep_in TEXT: this {case} --,NaN
12,I related her to Lennie in their inability to ...,{I},{related},{her},NaN,REASON: prep_in TEXT: {inability} -- REASON: p...,NaN
19,As I was racing through this novella in one si...,{I},could not {sing},{raptures},NaN,REASON: prep_to TEXT: {you} --,NaN
21,some of which topped my 5'3'' frame by a foot ...,a few {times},{topped},5'3 {frame},NaN,NaN,REASON: prep_by TEXT: a {foot} --
22,some of which topped my 5'3'' frame by a foot ...,{I},{topped},5'3 {frame},NaN,NaN,REASON: prep_by TEXT: a {foot} --
23,some of which topped my 5'3'' frame by a foot ...,{some} which,{topped},5'3 {frame},NaN,NaN,REASON: prep_by TEXT: a {foot} --
26,At least that's the nagging feeling I got from...,{I},{got},the nagging {feeling},NaN,REASON: prep_from TEXT: the {way} -- REASON: p...,REASON: prep_at TEXT: {least} --
27,At least that's the nagging feeling I got from...,a 't {art},{meant},{trouble},REASON: prep_like TEXT: {her} --,NaN,REASON: prep_for TEXT: a {man} --


In [29]:
print "number of relationships:",len(df)
print "number of relationships with at least one preposition:", len(df_with_prep)
# "number of relationships with at least two preposition: 89"
# "number of relationships with at least three preposition: 2"

number of relationships: 6727
number of relationships with at least one preposition: 1966


In [73]:
df_with_prep

,sentence,arg1,rel,arg2,arg1_prepositions,rel_prepositions,arg2_prepositions
2,"Crooks, the negro stable buck, had his bunk in the harness room; a little shed that leaned off the wall of the barn.",{Crooks},{had},{bunk},NaN,NaN,REASON: prep_in TEXT: the harness {room} --
5,Proof the incredibly sad ending is not hopeless is that on a reread I could still hope it would end differently.,{I},could still {hope},{it},NaN,REASON: prep_on TEXT: a {reread} --,NaN
9,"Dignity, yes, because in this case the dying was robbing his life of dignity.",{the} dying,was {robbing},{life} dignity,NaN,REASON: prep_in TEXT: this {case} --,NaN
12,I related her to Lennie in their inability to think and mourning what happens without any possibility of understanding why.,{I},{related},{her},NaN,REASON: prep_in TEXT: {inability} -- REASON: prep_to TEXT: {Lennie} --,NaN
19,"As I was racing through this novella in one sitting, I came to the last few pages and was stricken to realize that I could not sing raptures to you about the brotherhood of man.",{I},could not {sing},{raptures},NaN,REASON: prep_to TEXT: {you} --,NaN
21,some of which topped my 5'3'' frame by a foot or so and outweighed me by a good hundred pounds but despite that a few times I had to physically put myself on between them and a smaller child .,a few {times},{topped},5'3 {frame},NaN,NaN,REASON: prep_by TEXT: a {foot} --
22,some of which topped my 5'3'' frame by a foot or so and outweighed me by a good hundred pounds but despite that a few times I had to physically put myself on between them and a smaller child .,{I},{topped},5'3 {frame},NaN,NaN,REASON: prep_by TEXT: a {foot} --
23,some of which topped my 5'3'' frame by a foot or so and outweighed me by a good hundred pounds but despite that a few times I had to physically put myself on between them and a smaller child .,{some} which,{topped},5'3 {frame},NaN,NaN,REASON: prep_by TEXT: a {foot} --
26,"At least that's the nagging feeling I got from this story, from the way her character was handled, from the way it was repeatedly stated that a 'tart' like her meant trouble for a man.",{I},{got},the nagging {feeling},NaN,REASON: prep_from TEXT: the {way} -- REASON: prep_from TEXT: this {story} --,REASON: prep_at TEXT: {least} --
27,"At least that's the nagging feeling I got from this story, from the way her character was handled, from the way it was repeatedly stated that a 'tart' like her meant trouble for a man.",a 't {art},{meant},{trouble},REASON: prep_like TEXT: {her} --,NaN,REASON: prep_for TEXT: a {man} --


In [31]:
entity_versions = get_entity_versions(DATA_SET)

In [40]:
entity_versions_lists = []
for i in entity_versions.values():
    for j in i:
        entity_versions_lists.append(j)

#entity_versions_lists

In [71]:
col_rel_prep = "rel_prepositions"
col_arg2_prep = "arg2_prepositions"

import numpy as np
df_res = df_with_prep.replace(np.nan, '', regex=True)

#df_res = df_with_prep.copy()

ind_list = []
pd.set_option('display.max_colwidth', -1)
for index, row in df_res.iterrows():
    # lower case the letters
    for entity_of_interest in entity_versions_lists:
        if entity_of_interest in row["rel_prepositions"]:
            print str(row)
            ind_list.append(index)
        #if entity_of_interest in row["rel_prepositions"]:
        #    print row
#df_filtered = df[np.logical_or(df[col_rel_prep].isin(entity_versions_lists), df[col_arg2_prep].isin(entity_versions_lists))]

sentence             Lennie was the one that was putting a hold on his dreams.
arg1                 the {one}                                                
rel                  was {putting}                                            
arg2                 a {hold}                                                 
arg1_prepositions                                                             
rel_prepositions     REASON: prep_on TEXT: {dreams} --                        
arg2_prepositions                                                             
Name: 246, dtype: object
sentence             Lennie was the one that was putting a hold on his dreams.
arg1                 the {one}                                                
rel                  was {putting}                                            
arg2                 a {hold}                                                 
arg1_prepositions                                                             
rel_prepositions     REASON